### システム概要 - 仕様書

#### 1. **課題**
- **目標**: FXのトレードを毎日繰り返し、最終日の日本円での資産の最大化。
- **評価**: 以下のシナリオのうち、Jun_Aug_2017_without_commission、Jun_Aug_2017_with_commissionに提出いただいた最初の5回の平均と最大。

#### 2. **条件**
- **初期資産**: 1,000,000円
- **トレード頻度**: 平日に一日一度、FXレートを基に貨幣の取引が可能。
- **手数料**: 取引額のX％
- **取引可能な通貨**: JPY, USD, EUR, AUD, HKD
- **評価方法**: 最終日に全資産をJPYに変換し、その価値を評価。

#### 3. **問題セット**
各問題セットでは、以下の情報が提供されます：
- **シナリオ名**: Scenario name
- **取引開始日**: Start Date
- **取引終了日**: End Date
- **手数料**: 手数料率

例:
- TEST0: 2016-01-04 ～ 2016-01-11, 手数料 0%
- TEST1: 2016-01-04 ～ 2016-01-11, 手数料 0.1%
- Jun_Aug_2016_without_commission: 2016-06-01 ～ 2016-08-31, 手数料 0%
- Jun_Aug_2016_with_commission: 2016-06-01 ～ 2016-08-31, 手数料 0.1%
- Feb_Apr_2017_without_commission: 2017-02-01 ～ 2017-04-28, 手数料 0%
- Feb_Apr_2017_wit_commission: 2017-02-01 ～ 2017-04-28, 手数料 0.1%
- Jun_Aug_2017_without_commission: 2017-06-01 ～ 2017-08-31, 手数料 0%
- Jun_Aug_2017_with_commission: 2017-06-01 ～ 2017-08-31, 手数料 0.1%

Jun_Aug_2017_without_commissionとJun_Aug_2017_with_commissionにつきましては、評価で使用するために、当日皆様にお配りするuserIdを用いないとセッションを開始することはできません。

#### 4. **API 概要**
システムは以下のAPIを使用してトレードセッションを管理します。

1. **セッション開始**:
   - **概要**: 指定されたシナリオとUserIdに基づいて、トレードセッションを開始します。
   - **エンドポイント**: `POST /api/trade/start/<scenario>/<userId>`
   - **userId**: userIdは皆さんにご入場の際にお配りした6文字をお使いください。
   - **currentDate**: 現在の日付
   - **jpyBalance**: 現在の日本円での資産。
   - **currentcyToBalance**: 現在のそれぞれの貨幣の資産の量
   - **currentDateRate**: 現在の日付の為替レート
   - **complete**: セッションが終了しているかどうかを表すフラグ
   - **レスポンス例**:
     ```json
     {
       "sessionId": 0,
       "startDate": "2016-01-04",
       "endDate": "2016-01-11",
       "currentDate": "2016-01-04",
       "jpyBalance": 1000000,
       "currencyToBalance": {
         "JPY": 1000000
       },
       "currentDateRates": {
         "USD/JPY": 120.4,
         "EUR/JPY": 130.59
       },
       "complete": false
     }
     ```

2. **取引の実行**:
   - **概要**: セッションの現在日付において、指定された取引を実行します。取引は最終日まで繰り返されます。指定した量の貨幣をcurrencyFromからcurrencyToへ、為替レートcurrentDateRatesを使って変換します。
   
   - **エンドポイント**: `POST /api/trade/next`
   - **リクエスト例**:
     ```json
     {
       "sessionId": 0,
       "exchangeRequests": [
         {
           "currencyFrom": "JPY",
           "currencyTo": "USD",
           "amount": 1000
         },
         {
           "currencyFrom": "JPY",
           "currencyTo": "AUD",
           "amount": 1000
         }
       ]
     }
     ```
   - **レスポンス例**:
   - レスポンスは`POST /api/trade/start/<scenario>/<userId>`と同様だが、currencyToBalanceが先ほどのリクエストに基づいて変更され、currentDateは翌日に、currentDateRatesが翌日の為替レートに、jpyBalanceが翌日の為替レートと現在の保有資産に基づいてアップデートされている。
     ```json
     {
       "sessionId": 0,
       "startDate": "2016-01-04",
       "endDate": "2016-01-11",
       "currentDate": "2016-01-05",
       "jpyBalance": 999969.0392754745,
       "currencyToBalance": {
         "JPY": 998000,
         "USD": 8.30564784053156,
         "AUD": 11.415525114155253
       },
       "currentDateRates": {
         "USD/JPY": 119.27,
         "EUR/JPY": 129.17
       },
       "complete": false
     }
     ```
    - これをcurrentDate == endDate になるまで繰り返す。その際、complete == trueとなっている。

3. **FxRateの参照**:
   - **エンドポイント**:
     - `GET /api/rate/{date}`:
   - **レスポンス例**:
     ```json
     {
         "USD/JPY": 120.4,
         "EUR/JPY": 130.59
     }
     ```
**その他詳細**:
- exchangeRequestsのamountが所有資産のcurrencyFromより多い場合、所有資産の全てをcurrencyToに変換する。
- exchangeRequestsのamountが0より小さい場合、requestは無視される。
- rateのAPIは2002-04-01から2023-06-01の間の為替レートを返す。


In [38]:
# import library
%matplotlib inline
import requests
import copy
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime, timedelta
from matplotlib import ticker

In [40]:
# Define common variables
CURRENCY_LIST = ['JPY','USD','AUD','HKD','EUR']
JPY_PAIR_LIST =[currency + '/JPY'  for currency in CURRENCY_LIST if currency != 'JPY']
CURRENCY_PAIR = [f"{currency1}/{currency2}" for currency1 in CURRENCY_LIST for currency2 in CURRENCY_LIST if currency1 != currency2]
TESTCASE = 'Feb_Apr_2017_without_commission: 2017-02-01 ～ 2017-04-28, 手数料 0%' # 編集するとこ
USERID = '3a6qdr'
BASE_URL = 'http://os3-389-27987.vs.sakura.ne.jp/'
START_URL = BASE_URL + 'api/trade/start/{}/{}'
NEXT_URL = BASE_URL + 'api/trade/next'
RATE_URL = BASE_URL + 'api/rate/{}'

In [41]:
# Define common functions
def get_last_n_weekdays(date_str, n):
    given_date = datetime.strptime(date_str, '%Y-%m-%d').date()
    weekdays = []

    current_date = given_date
    while len(weekdays) < n:
        if current_date.weekday() < 5:  # Monday is 0 and Friday is 4
            weekdays.append(current_date.strftime('%Y-%m-%d'))
        current_date -= timedelta(days=1)

    return weekdays[::-1]

def init_rates_last_n_weekdays(date_str, n):
    dates = get_last_n_weekdays(date_str, n)
    currency_pair_to_rates = {currency_pair: [] for currency_pair in CURRENCY_PAIR}
    for date in dates:
        response = requests.get(RATE_URL.format(date))
        currency_pair_to_rate = response.json()
        for currency_pair, rate in currency_pair_to_rate.items(): currency_pair_to_rates[currency_pair].append(rate)
    return currency_pair_to_rates

def update_rates(currency_pair_to_rates, next_date_rates):
    for currency_pair, rate in next_date_rates.items(): currency_pair_to_rates[currency_pair].append(rate)

def get_currency_pairs(currency_from):
    return [currency + '/' + currency_from for currency in CURRENCY_LIST if currency != currency_from]

def plot_chart(axes, rates, today=None, label=None):
    axes.xaxis.set_major_locator(ticker.MultipleLocator(10))
    if today:
        x = get_last_n_weekdays(today, len(rates))
    else:
        x = range(len(rates))
    axes.plot(x, rates, label=label)

def get_dma(rates, x):
    return [sum(rates[max(0, i-x):i+1]) / len(rates[max(0, i-x):i+1]) for i in range(len(rates))]

固定金額積立

In [ ]:
# Example with changing fixed JPY amount to USD, AUD
# 固定値積立
def fixed_amount_from_jpy_to_usd(amount):
    return [{
            'currencyFrom': 'JPY',
            'currencyTo': 'USD',
            'amount': amount
        }, {
            'currencyFrom': 'JPY',
            'currencyTo': 'AUD',
            'amount': amount
        }]

In [ ]:
TESTCASE = 'TEST0'

response = requests.post(START_URL.format(TESTCASE, USERID))
session_info = response.json()
session_id = session_info['sessionId']
is_complete = session_info['complete']
start_date = session_info['startDate']
current_date = session_info['currentDate']
currency_to_balance = session_info['currencyToBalance']
currency_pair_to_rates = init_rates_last_n_weekdays(start_date, 10)

while not is_complete:
    exchange_requests = fixed_amount_from_jpy_to_usd(1000000000)
    request_body_next = {
        'sessionId': session_id,
        'exchangeRequests': exchange_requests
    }
    if len(exchange_requests) > 0: print('current date: ' + current_date + ', exchange_requests' + str(exchange_requests))
    response = requests.post(NEXT_URL, json=request_body_next)
    next_info = response.json()
    update_rates(currency_pair_to_rates, next_info['currentDateRates'])
    current_date = next_info['currentDate']
    currency_to_balance = next_info['currencyToBalance']
    is_complete = next_info['complete']

result = next_info['jpyBalance']
print(result)

逆張りの例

In [ ]:
# 逆張り戦略の例
# 所持している資産の通貨が{x}日間連続で別の通貨に対して上昇しているとき、その通貨に{y}%変更する
# 例えば、USD/JPYが3日間連続で減少しているとき、[130, 129, 128]、円はドルに対して価値が上昇しているので、円をドルに変更する
CURRENCY_LIST = ['JPY','USD','AUD','HKD','EUR']

def contranian_strategy(currency_pair_to_rates, currency_to_balance, x, y):
    exchange_requests = []
    for currency, balance in currency_to_balance.items():
        if balance > 0:
            currency_pairs = get_currency_pairs(currency)
            for currency_pair in currency_pairs:
                rates = currency_pair_to_rates[currency_pair][-x:]
                if all(rates[i] > rates[i + 1] for i in range(len(rates) - 1)):
                    print(currency_pair + ' rates for past x days: ' + str(rates))
                    currency_pair_to_exchange = currency_pair
                    exchange_requests.append({
                        'currencyFrom': currency,
                        'currencyTo': currency_pair[:3],
                        'amount': balance * y / 100.0
                    })
                    break
    return exchange_requests

In [ ]:
TESTCASE = 'Jun_Aug_2016_without_commission'

response = requests.post(START_URL.format(TESTCASE, USERID))
session_info = response.json()
session_id = session_info['sessionId']
is_complete = session_info['complete']
start_date = session_info['startDate']
current_date = session_info['currentDate']
currency_to_balance = session_info['currencyToBalance']
currency_pair_to_rates = init_rates_last_n_weekdays(start_date, 10)

while not is_complete:
    exchange_requests = contranian_strategy(currency_pair_to_rates, currency_to_balance, 5, 30)
    request_body_next = {
        'sessionId': session_id,
        'exchangeRequests': exchange_requests
    }
    if len(exchange_requests) > 0: print('current date: ' + current_date + ', exchange_requests' + str(exchange_requests))
    response = requests.post(NEXT_URL, json=request_body_next)
    next_info = response.json()
    update_rates(currency_pair_to_rates, next_info['currentDateRates'])
    current_date = next_info['currentDate']
    currency_to_balance = next_info['currencyToBalance']
    is_complete = next_info['complete']

result = next_info['jpyBalance']
print(result)

ゴールデンクロス・デッドクロスの例

In [ ]:
# ゴールデンクロス・デッドクロスの例
# 全資産をJPYを除く各通貨に均等に割り当て、x日移動平均がy日移動平均を上回る場合は上昇トレンド、下回る場合は下降トレンドと判断し、その通貨に変更する
# 長いトレンドと短いトレンドを比較して
def golden_and_death_cross_strategy(currency_pair_to_rates, currency_to_balance, x, y):
    exchange_requests = []
    trend = {}
    # ポジションを持っていない通貨を計算
    currencies_without_position = [currency for currency, balance in currency_to_balance.items() if balance == 0]

    # それぞれの通貨についてトレンドを計算
    for currency, balance in currency_to_balance.items():
        if currency == 'JPY':
            continue
        currency_pair = currency + '/JPY'
        if sum(currency_pair_to_rates[currency_pair][-x:])/x > sum(currency_pair_to_rates[currency_pair][-y:])/y:
            # 上昇トレンドと判断
            if balance == 0:
                # (保有する円/ポジションを持っていない通貨の数)を変更
                exchange_requests.append({
                        'currencyFrom': 'JPY',
                        'currencyTo': currency,
                        'amount': currency_to_balance['JPY'] / len(currencies_without_position)
                    })
        else:
            # 下降トレンドと判断
            if  balance > 0:
                # ポジションを解消
                exchange_requests.append({
                        'currencyFrom': currency,
                        'currencyTo': 'JPY',
                        'amount': currency_to_balance[currency]
                    })
    return exchange_requests

In [ ]:
TESTCASE = 'Jun_Aug_2016_without_commission'
X = 5
Y = 10

response = requests.post(START_URL.format(TESTCASE, USERID))
session_info = response.json()
session_id = session_info['sessionId']
is_complete = session_info['complete']
start_date = session_info['startDate']
current_date = session_info['currentDate']
currency_to_balance = session_info['currencyToBalance']
currency_pair_to_rates = init_rates_last_n_weekdays(start_date, 10)

while not is_complete:
    exchange_requests = golden_and_death_cross_strategy(currency_pair_to_rates, currency_to_balance, X, Y)
    request_body_next = {
        'sessionId': session_id,
        'exchangeRequests': exchange_requests
    }
    if len(exchange_requests) > 0: print('current date: ' + current_date + ', exchange_requests' + str(exchange_requests))
    response = requests.post(NEXT_URL, json=request_body_next)
    next_info = response.json()
    update_rates(currency_pair_to_rates, next_info['currentDateRates'])
    current_date = next_info['currentDate']
    currency_to_balance = next_info['currencyToBalance']
    is_complete = next_info['complete']

result = next_info['jpyBalance']
print(current_date, result)

In [ ]:
figure, ax = plt.subplots(figsize=(30,10))
ax.grid()
currency_pair = 'USD/JPY'
plot_chart(ax, currency_pair_to_rates[currency_pair], today=current_date, label=currency_pair)
plot_chart(ax, get_dma(currency_pair_to_rates[currency_pair], X), today=current_date, label=currency_pair + ' {}DMA'.format(X))
plot_chart(ax, get_dma(currency_pair_to_rates[currency_pair], Y), today=current_date, label=currency_pair + ' {}DMA'.format(Y))
plt.legend()
plt.show()

## 実行

ゴールデンクロス・デッドクロス

短期：5日　長期:5×i日　の場合

In [42]:
# ゴールデンクロス・デッドクロスの例
# 全資産をJPYを除く各通貨に均等に割り当て、x日移動平均がy日移動平均を上回る場合は上昇トレンド、下回る場合は下降トレンドと判断し、その通貨に変更する
def golden_and_death_cross_strategy(currency_pair_to_rates, currency_to_balance, x, y):
    exchange_requests = []
    trend = {}
    # ポジションを持っていない通貨を計算
    currencies_without_position = [currency for currency, balance in currency_to_balance.items() if balance == 0]

    # それぞれの通貨についてトレンドを計算
    for currency, balance in currency_to_balance.items():
        if currency == 'JPY':
            continue
        currency_pair = currency + '/JPY'
        if sum(currency_pair_to_rates[currency_pair][-x:])/x > sum(currency_pair_to_rates[currency_pair][-y:])/y:
            # 上昇トレンドと判断
            if balance == 0:
                # (保有する円/ポジションを持っていない通貨の数)を変更
                exchange_requests.append({
                        'currencyFrom': 'JPY',
                        'currencyTo': currency,
                        'amount': currency_to_balance['JPY'] / len(currencies_without_position)
                    })
        else:
            # 下降トレンドと判断
            if  balance > 0:
                # ポジションを解消
                exchange_requests.append({
                        'currencyFrom': currency,
                        'currencyTo': 'JPY',
                        'amount': currency_to_balance[currency]
                    })
    return exchange_requests

In [ ]:
# ゴールデンクロスデッドクロスの移動平均を変えてみる

TESTCASE = 'Jun_Aug_2016_without_commission'
result_list_gd = []

for i in (2,15):
    X = 5 
    Y = X * i
    response = requests.post(START_URL.format(TESTCASE, USERID))
    session_info = response.json()
    session_id = session_info['sessionId']
    is_complete = session_info['complete']
    start_date = session_info['startDate']
    current_date = session_info['currentDate']
    currency_to_balance = session_info['currencyToBalance']
    currency_pair_to_rates = init_rates_last_n_weekdays(start_date, 10)

    while not is_complete:
        exchange_requests = golden_and_death_cross_strategy(currency_pair_to_rates, currency_to_balance, X, Y)
        request_body_next = {
            'sessionId': session_id,
            'exchangeRequests': exchange_requests
        }
        if len(exchange_requests) > 0: print('current date: ' + current_date + ', exchange_requests' + str(exchange_requests))
        response = requests.post(NEXT_URL, json=request_body_next)
        next_info = response.json()
        update_rates(currency_pair_to_rates, next_info['currentDateRates'])
        current_date = next_info['currentDate']
        currency_to_balance = next_info['currencyToBalance']
        is_complete = next_info['complete']

    result = next_info['jpyBalance']
    # print(current_date, result)
    result_list_gd.append(result)

In [ ]:
print(result_list_gd)

短期：3日　長期:3×i日　の場合

In [44]:
# これを提出
# ゴールデンクロスデッドクロスの移動平均を変えてみる

TESTCASE = 'Feb_Apr_2017_without_commission: 2017-02-01 ～ 2017-04-28, 手数料 0%'

for _ in range(5):
    X = 3
    Y = X * 2
    response = requests.post(START_URL.format(TESTCASE, USERID))
    session_info = response.json()
    session_id = session_info['sessionId']
    is_complete = session_info['complete']
    start_date = session_info['startDate']
    current_date = session_info['currentDate']
    currency_to_balance = session_info['currencyToBalance']
    currency_pair_to_rates = init_rates_last_n_weekdays(start_date, 10)

    while not is_complete:
        exchange_requests = golden_and_death_cross_strategy(currency_pair_to_rates, currency_to_balance, X, Y)
        request_body_next = {
            'sessionId': session_id,
            'exchangeRequests': exchange_requests
        }
        if len(exchange_requests) > 0: print('current date: ' + current_date + ', exchange_requests' + str(exchange_requests))
        response = requests.post(NEXT_URL, json=request_body_next)
        next_info = response.json()
        update_rates(currency_pair_to_rates, next_info['currentDateRates'])
        current_date = next_info['currentDate']
        currency_to_balance = next_info['currencyToBalance']
        is_complete = next_info['complete']

    result = next_info['jpyBalance']
    print(current_date, result)

KeyError: 'sessionId'

短期：7日　長期:7×i日　の場合

In [ ]:
# ゴールデンクロスデッドクロスの移動平均を変えてみる

TESTCASE = 'Jun_Aug_2016_without_commission'
result_list_gd_7 = []

for i in (2,12):
    X = 7
    Y = X * i
    response = requests.post(START_URL.format(TESTCASE, USERID))
    session_info = response.json()
    session_id = session_info['sessionId']
    is_complete = session_info['complete']
    start_date = session_info['startDate']
    current_date = session_info['currentDate']
    currency_to_balance = session_info['currencyToBalance']
    currency_pair_to_rates = init_rates_last_n_weekdays(start_date, 10)

    while not is_complete:
        exchange_requests = golden_and_death_cross_strategy(currency_pair_to_rates, currency_to_balance, X, Y)
        request_body_next = {
            'sessionId': session_id,
            'exchangeRequests': exchange_requests
        }
        if len(exchange_requests) > 0: print('current date: ' + current_date + ', exchange_requests' + str(exchange_requests))
        response = requests.post(NEXT_URL, json=request_body_next)
        next_info = response.json()
        update_rates(currency_pair_to_rates, next_info['currentDateRates'])
        current_date = next_info['currentDate']
        currency_to_balance = next_info['currencyToBalance']
        is_complete = next_info['complete']

    result = next_info['jpyBalance']
    # print(current_date, result)
    result_list_gd_7.append(result)

In [ ]:
print(result_list_gd_7)

長期トレンドと短期トレンドの期間はそんなに差がない方がいい

逆張り

In [ ]:
# 逆張り戦略


# 所持している資産の通貨が{x}日間連続で別の通貨に対して上昇しているとき、その通貨に{y}%変更する
# 例えば、USD/JPYが3日間連続で減少しているとき、[130, 129, 128]、円はドルに対して価値が上昇しているので、円をドルに変更する
CURRENCY_LIST = ['JPY','USD','AUD','HKD','EUR']

def contranian_strategy(currency_pair_to_rates, currency_to_balance, x, y):
    exchange_requests = []
    for currency, balance in currency_to_balance.items():
        if balance > 0:
            currency_pairs = get_currency_pairs(currency)
            for currency_pair in currency_pairs:
                rates = currency_pair_to_rates[currency_pair][-x:]
                if all(rates[i] > rates[i + 1] for i in range(len(rates) - 1)):
                    print(currency_pair + ' rates for past x days: ' + str(rates))
                    currency_pair_to_exchange = currency_pair
                    exchange_requests.append({
                        'currencyFrom': currency,
                        'currencyTo': currency_pair[:3],
                        'amount': balance * y / 100.0
                    })
                    break
    return exchange_requests

In [ ]:
TESTCASE_list = ['TEST0', 'TEST1', 'Jun_Aug_2016_without_commission', 'Jun_Aug_2016_with_commission']
result_list = []
for TESTCASE in TESTCASE_list:
    response = requests.post(START_URL.format(TESTCASE, USERID))
    session_info = response.json()
    session_id = session_info['sessionId']
    is_complete = session_info['complete']
    start_date = session_info['startDate']
    current_date = session_info['currentDate']
    currency_to_balance = session_info['currencyToBalance']
    currency_pair_to_rates = init_rates_last_n_weekdays(start_date, 10)

    while not is_complete:
        exchange_requests = contranian_strategy(currency_pair_to_rates, currency_to_balance, 5, 30)
        request_body_next = {
            'sessionId': session_id,
            'exchangeRequests': exchange_requests
        }
        if len(exchange_requests) > 0: print('current date: ' + current_date + ', exchange_requests' + str(exchange_requests))
        response = requests.post(NEXT_URL, json=request_body_next)
        next_info = response.json()
        update_rates(currency_pair_to_rates, next_info['currentDateRates'])
        current_date = next_info['currentDate']
        currency_to_balance = next_info['currencyToBalance']
        is_complete = next_info['complete']

    result = next_info['jpyBalance']
    result_list.append(result)

In [ ]:
print(result_list)

USD,AUD

In [ ]:
# Example with changing fixed JPY amount to USD, AUD
# 固定値積立
def fixed_amount_from_jpy_to_usd(amount):
    return [{
            'currencyFrom': 'JPY',
            'currencyTo': 'USD',
            'amount': amount
        }, {
            'currencyFrom': 'JPY',
            'currencyTo': 'AUD',
            'amount': amount
        }]

In [ ]:
TESTCASE_list = ['TEST0', 'TEST1', 'Jun_Aug_2016_without_commission', 'Jun_Aug_2016_with_commission']
result_list_USD_AUD = []
for TESTCASE in TESTCASE_list:
    response = requests.post(START_URL.format(TESTCASE, USERID))
    session_info = response.json()
    session_id = session_info['sessionId']
    is_complete = session_info['complete']
    start_date = session_info['startDate']
    current_date = session_info['currentDate']
    currency_to_balance = session_info['currencyToBalance']
    currency_pair_to_rates = init_rates_last_n_weekdays(start_date, 10)

    while not is_complete:
        exchange_requests = fixed_amount_from_jpy_to_usd(1000000000)
        request_body_next = {
            'sessionId': session_id,
            'exchangeRequests': exchange_requests
        }
        if len(exchange_requests) > 0: print('current date: ' + current_date + ', exchange_requests' + str(exchange_requests))
        response = requests.post(NEXT_URL, json=request_body_next)
        next_info = response.json()
        update_rates(currency_pair_to_rates, next_info['currentDateRates'])
        current_date = next_info['currentDate']
        currency_to_balance = next_info['currencyToBalance']
        is_complete = next_info['complete']

    result = next_info['jpyBalance']
    result_list_USD_AUD.append(result)

In [ ]:
print(result_list_USD_AUD)

手数料なしの方向で

## Transformerを使ってみる

In [26]:
import requests
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler


In [30]:
USERID = '3a6qdr'
TESTCASE = 'Jun_Aug_2016_without_commission'
BASE_URL = 'http://os3-389-27987.vs.sakura.ne.jp/'
START_URL = BASE_URL + 'api/trade/start/{}/{}'
RATE_URL = BASE_URL + 'api/rate/{}'
CURRENCY_LIST = ['JPY','USD','AUD','HKD','EUR']
JPY_PAIR_LIST = [currency + '/JPY'  for currency in CURRENCY_LIST if currency != 'JPY']

response = requests.get(START_URL.format(USERID, TESTCASE))
if response.status_code == 200:
    data = response.json()
    # レートデータを取得
    rate_data = []
    for pair in JPY_PAIR_LIST:
        rate_response = requests.get(RATE_URL.format(pair))
        if rate_response.status_code == 200:
            rate_data.append(rate_response.json())
    # データの加工処理を行う（ここは具体的なデータ構造に応じて実装）


In [34]:
USERID = '3a6qdr'
TESTCASE = 'Jun_Aug_2016_without_commission'
BASE_URL = 'http://os3-389-27987.vs.sakura.ne.jp/'
START_URL = BASE_URL + 'api/trade/start/{}/{}'
RATE_URL = BASE_URL + 'api/rate/{}'
CURRENCY_LIST = ['JPY','USD','AUD','HKD','EUR']
JPY_PAIR_LIST = [currency + '/JPY' for currency in CURRENCY_LIST if currency != 'JPY']

# スタートデータを取得
response = requests.post(START_URL.format(USERID, TESTCASE), json={"param1": "value1", "param2": "value2"})
if response.status_code == 200:
    data = response.json()
    print("Start Data:", data)  # デバッグ用出力

    # レートデータを取得
    rate_data = []  # rate_dataの定義を確認
    for pair in JPY_PAIR_LIST:
        rate_response = requests.get(RATE_URL.format(pair))
        if rate_response.status_code == 200:
            rate_data.append(rate_response.json())
        else:
            print(f"Failed to get rate for {pair}: {rate_response.status_code}")
    
    print("Rate Data:", rate_data)  # デバッグ用出力

    # rate_dataのサイズを確認
    if len(rate_data) > 0:
        # データの加工処理を行う（ここは具体的なデータ構造に応じて実装）
        # 例: データの整形、スケーリングなど
        pass
    else:
        print("No rate data was retrieved.")
else:
    print(f"Failed to start trading session: {response.status_code}")


Failed to start trading session: 500


In [28]:
# スケーリング
scaler = MinMaxScaler()
rate_data_scaled = scaler.fit_transform(rate_data)

# データセットの作成
PREDICTION_DAYS = 11
seq_length = 30

class TimeSeriesDataset(Dataset):
    def __init__(self, data, seq_length):
        self.data = torch.FloatTensor(data)
        self.seq_length = seq_length

    def __len__(self):
        return len(self.data) - self.seq_length - (PREDICTION_DAYS - 1)

    def __getitem__(self, index):
        return (self.data[index:index+self.seq_length, :-1], 
                self.data[index+self.seq_length+(PREDICTION_DAYS-1), -1])

train_dataset = TimeSeriesDataset(rate_data_scaled, seq_length)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


NameError: name 'rate_data' is not defined

In [ ]:
class FXTransformerModel(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, num_pairs):
        super(FXTransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim * num_pairs, d_model)
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead), num_layers)
        self.fc = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)
        x = self.transformer(x)
        x = x[-1]
        return self.fc(x).squeeze()


In [ ]:
input_dim = len(JPY_PAIR_LIST)
d_model = 64
nhead = 4
num_layers = 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = FXTransformerModel(input_dim, d_model, nhead, num_layers, num_pairs=len(JPY_PAIR_LIST)).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


In [ ]:
model.eval()
predictions = []
with torch.no_grad():
    for batch_x, _ in train_loader:
        batch_x = batch_x.to(device)
        output = model(batch_x).to("cpu")
        predictions.append(output.item())

# 予測結果を逆スケーリング
test_features = rate_data_scaled[seq_length:-(PREDICTION_DAYS-1)]
predictions_array = np.array(predictions).reshape(-1, 1)

predictions_unscaled = scaler.inverse_transform(
    np.column_stack([test_features, predictions_array]))[:, -1]


In [ ]:
print("予測収支:", predictions_unscaled)
